In [55]:
!pip install -q pypdf
!pip install -q python-dotenv
!pip install langchain
!pip install llama-index-embeddings-langchain
!pip install transformers
!pip install -q llama-index
%pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-llama-cpp

  Using cached packaging-23.2-py3-none-any.whl (53 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.24.0 requires anyio<4,>=3.1.0, but you have anyio 4.3.0 which is incompatible.
tensorflow 2.15.0 requires wrapt<1.15,>=1.11.0, but you have wrapt 1.16.0 which is incompatible.


TO make use of GPU with cpu this is the installation comand

In [57]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir

In [60]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext,PromptTemplate


**SimpleDirectoryReader** : Is used for reding the documents and storing into vectors.

In [61]:
# documents = SimpleDirectoryReader("/content/book.pdf").load_data()
doc = SimpleDirectoryReader(input_files=['/content/book.pdf'])
documents = doc.load_data()

In [62]:
import torch
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',

    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /tmp/llama_index/models/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:   

To convert text into emdebbing into a vector stores we need to use sentence transformeers.

In [63]:
!pip -q install sentence-transformers

**Why did i use thenlper/gte-large embedding models?**

1.   Answer: in Huggingface leaderboard for text embeddings gte-large is better then other sentence transformer.




---



In [48]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model = HuggingFaceEmbeddings(
  model_name="thenlper/gte-large"
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Here we need to create a **service contex**t as we are using our own customize llm dy default lamaindex make use of openai

In [69]:
service_context = ServiceContext.from_defaults(
    chunk_size=256,
    llm=llm,#mistral7b
    embed_model=embed_model#gtelarge
)

<ipython-input-69-7e47785a9da0>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


We can index the documents by vector store index

In [70]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

How does the **query engine** works **interally**

1.   It takes the query

2.   It create the embedding using owr embedding
3.   Searches in the vector space into the document which is the closest and revelent in the content for the given query ,It gets its and attached as a prompt.
4. once it get atatched as a prompt it send to llm as a response.[link text](https://)



In [72]:
query_engine = index.as_query_engine()
response = query_engine.query("What is cancer?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    5558.50 ms
llama_print_timings:      sample time =      73.56 ms /   126 runs   (    0.58 ms per token,  1712.98 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   84078.10 ms /   126 runs   (  667.29 ms per token,     1.50 tokens per second)
llama_print_timings:       total time =   84613.97 ms /   127 tokens


In [76]:
response = query_engine.query("What is cancer?")
print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    5558.50 ms
llama_print_timings:      sample time =      73.65 ms /   126 runs   (    0.58 ms per token,  1710.77 tokens per second)
llama_print_timings: prompt eval time =    6925.87 ms /   567 tokens (   12.21 ms per token,    81.87 tokens per second)
llama_print_timings:        eval time =   82047.60 ms /   125 runs   (  656.38 ms per token,     1.52 tokens per second)
llama_print_timings:       total time =   89521.76 ms /   692 tokens


 Cancer is a group of diseases associated with abnormal growth of cells. Without any check, the disease may keep on progressing ultimately leading to pre-mature death. They can arise anywhere in the body and can affect people from all age groups, socio-economic strata and race. Cancer is the leading cause of morbidity and mortality in the world. According to data by International Agency for Research on Cancer, there were 141 lakh new cancer cases, 82 lakh cancer deaths and 326 lakh people living with cancer in 2012 worldwide.


To ask the query in infine loop we can ask like this.

In [ ]:
while True:
  query=input()
  response = query_engine.query(query)
  print(response)